In [6]:
## 1.0. setup 

In [7]:
!conda install -c anaconda swig -y 

Solving environment: done

# All requested packages already installed.



In [8]:
%%bash

which conda
which python
conda --version

g++ --version 
cmake --version 
python --version 
swig -version 

python -c "import numpy; print(numpy.get_include())"

/opt/conda/bin/conda
/opt/conda/bin/python
conda 4.12.0
g++ (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

cmake version 3.16.3

CMake suite maintained and supported by Kitware (kitware.com/cmake).
Python 3.7.12

SWIG Version 4.0.2

Compiled with x86_64-conda-linux-gnu-c++ [x86_64-conda-linux-gnu]

Configured options: +pcre

Please see http://www.swig.org for reporting bugs and further information
/opt/conda/lib/python3.7/site-packages/numpy/core/include


In [9]:
## 2.0. 

In [10]:
%%bash 
git clone https://github.com/jmake/PyPhysX
tar -xf PyPhysX/Swig/install.tar 
pwd
ls -la 
#ls PyPhysX
#ls -la home

/kaggle/working
total 24
drwxr-xr-x 5 root root 4096 Jun  2 10:02 .
drwxr-xr-x 5 root root 4096 Jun  2 09:57 ..
drwxr-xr-x 2 root root 4096 Jun  2 09:57 .virtual_documents
drwxr-xr-x 6 root root 4096 Jun  2 10:02 PyPhysX
---------- 1 root root  263 Jun  2 09:57 __notebook_source__.ipynb
drwxr-xr-x 3 root root 4096 Jun  2 10:02 home


Cloning into 'PyPhysX'...


In [11]:
## 3.0 

In [12]:
%%writefile /kaggle/working/PyPhysX/Swig/CMakeLists.txt
cmake_minimum_required(VERSION 3.16)
project(simplest)


## Wrapper.Python
#set(... Release)
find_package(Python3 COMPONENTS Interpreter Development REQUIRED)
include_directories(${Python3_INCLUDE_DIRS})

## numpy.get_include() 
include_directories(${NUMPY_GET_INCLUDE})

## Wrapper.Python
find_package(SWIG REQUIRED)
include(${SWIG_USE_FILE})
#include (UseSWIG)

#include_directories("/home/jovyan/work/PyPhysX/Swig")
include_directories(${PROJECT_SOURCE_DIR})
message("PROJECT_SOURCE_DIR:${PROJECT_SOURCE_DIR}")
add_library(LibraryName SHARED physx.cpp)

set_property(SOURCE swigConfigurationFile.i PROPERTY CPLUSPLUS ON)
swig_add_library(SWIG_fileName LANGUAGE python SOURCES swigConfigurationFile.i)
#swig_link_libraries(SWIG_fileName LibraryName ${Python3_LIBRARIES})


## 0.0. Setting... 
set(CMAKE_CXX_STANDARD 11)
set(CMAKE_CXX_STANDARD_REQUERIRED ON)
set(CMAKE_BUILD_TYPE Release)
set(CMAKE_CXX_FLAGS "${CMAKE_CXX_FLAGS} -Wl,--no-as-needed -ldl -pthread ")


## 1.0. HEADERS 
include_directories("${PHYSX_ROOT_DIR}/linux/PhysX/include")    #  
include_directories("${PHYSX_ROOT_DIR}/linux/PxShared/include") # foundation


## 2.0. LIBRARIES 
#link_directories("${PHYSX_ROOT_DIR}/linux/PhysXSO/bin/linux.clang/release") 

set(DUMMIES 
    PhysXExtensions 
    PhysX 
    PhysXPvdSDK 
    PhysXVehicle 
    PhysXCharacterKinematic 
    PhysXCooking 
    PhysXCommon 
    PhysXFoundation 
   )

foreach(DUMMY ${DUMMIES})
   list(APPEND LIBRARIES "lib${DUMMY}_64.so")
endforeach()

swig_link_libraries(SWIG_fileName LibraryName ${Python3_LIBRARIES}) # ${LIBRARIES}

Overwriting /kaggle/working/PyPhysX/Swig/CMakeLists.txt


In [13]:
%%bash 

CHECKING()
{ 
  echo "LIBRARIES"
  ls -la ${PHYSX_ROOT_DIR}/linux/PhysXSO/bin/linux.clang/release/
  
  echo "INCLUDES:"
  ls ${PHYSX_ROOT_DIR}/linux/PhysX/include/PxConfig.h
  ls ${PHYSX_ROOT_DIR}/linux/PxShared/include/foundation/Px.h

  find ${NUMPY_GET_INCLUDE} | grep arrayobject.h
}


SIMULATION_RUN()
{
  cd $SWIG_PATH
  
  rm -rf BUILD
  mkdir BUILD
  cd BUILD

  cmake .. \
    -DCMAKE_RULE_MESSAGES=OFF \
    -DPHYSX_ROOT_DIR=${PHYSX_ROOT_DIR} \
    -DNUMPY_GET_INCLUDE=${NUMPY_GET_INCLUDE}

  make

  #bash ../link.txt >> /dev/null
}


RELINK()
{
    cd $SWIG_PATH/BUILD
    
    find . | grep link.txt 

    #more 
    LIBS=$(ls ${PHYSX_ROOT_DIR}/linux/PhysXSO/bin/linux.clang/release/*_64.so)
    LINK=$(cat ./CMakeFiles/SWIG_fileName.dir/link.txt)
    echo ${LINK} ${LIBS} > patch.txt 
    #cat patch.txt
        
    rm _SWIG_fileName.so     
    bash patch.txt    
    export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:${PHYSX_ROOT_DIR}/linux/PhysXSO/bin/linux.clang/release/
    ldd _SWIG_fileName.so  
}

TESTING()
{
  cd $SWIG_PATH

  export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:${PHYSX_ROOT_DIR}/linux/PhysXSO/bin/linux.clang/release/
  export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:${SWIG_PATH}/BUILD

  python test.py
}

## 
NUMPY_GET_INCLUDE=$(python -c "import numpy; print(numpy.get_include())")
SWIG_PATH=${PWD}/PyPhysX/Swig
PHYSX_ROOT_DIR=${PWD}/home/jovyan/work/physx/install/

## a. 
CHECKING

## b. 
SIMULATION_RUN # libLibraryName.so swig_example.py  
RELINK # _SWIG_fileName.so

## c.
TESTING

LIBRARIES
total 13216
drwxr-xr-x 2 jupyter root    4096 May 19 16:41 .
drwxr-xr-x 3 jupyter root    4096 May 19 08:51 ..
-rw-r--r-- 1 jupyter root  197216 May 19 16:39 libPhysXCharacterKinematic_64.so
-rw-r--r-- 1 jupyter root  271572 May 19 08:49 libPhysXCharacterKinematic_static_64.a
-rw-r--r-- 1 jupyter root 1653344 May 19 16:34 libPhysXCommon_64.so
-rw-r--r-- 1 jupyter root  294248 May 19 16:38 libPhysXCooking_64.so
-rw-r--r-- 1 jupyter root 1826768 May 19 16:41 libPhysXExtensions_64.so
-rw-r--r-- 1 jupyter root 2658990 May 19 08:51 libPhysXExtensions_static_64.a
-rw-r--r-- 1 jupyter root   89536 May 19 16:31 libPhysXFoundation_64.so
-rw-r--r-- 1 jupyter root  333856 May 19 16:31 libPhysXPvdSDK_64.so
-rw-r--r-- 1 jupyter root  512040 May 19 08:41 libPhysXPvdSDK_static_64.a
-rw-r--r-- 1 jupyter root  853352 May 19 16:39 libPhysXVehicle_64.so
-rw-r--r-- 1 jupyter root 1165516 May 19 08:49 libPhysXVehicle_static_64.a
-rw-r--r-- 1 jupyter root 3641520 May 19 16:38 libPhysX_64.so
INCLUD

PROJECT_SOURCE_DIR:/kaggle/working/PyPhysX/Swig


In [14]:
!date

Thu Jun  2 10:02:25 UTC 2022
